## 4.10 Part 1 : Instacart

#### Content:

#### 1. Import Libraries and Dataframe

#### 2. Address PII in the Dataframe

#### 3. Create Regional Segmentation

#### 4. Create Exclusion Flag for Low-Activity Customers

#### 1. Import Libraries and Dataframe

In [1]:
# Import Library

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
# Import dataframe

path = r'/Users/queency_yustiawan/Study/CF/data analyst course/immersion/A4 exercise/Instacart Basket Analysis'

df = pd.read_pickle (os.path.join (path, '02 Data', 'Prepared Data', 'orders_products_customers_rev.pkl'))

In [4]:
# Checking output

df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,order_interval_days,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,first_name,surname,gender,state,age,date_joined,number_of_dependants,family_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequenct customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequenct customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequenct customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequenct customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequenct customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [5]:
df.shape

(32404859, 33)

#### 2. Address PII in the Dataframe

##### The dataframe contains the customers' first and last name, to ensure data privacy of the customers these two columns will be removed.

In [7]:
# Dropping 'first_name' and 'surname' column

df = df.drop(columns = ['first_name', 'surname'])

In [8]:
# Checking output

df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,order_interval_days,product_id,add_to_cart_order,reordered,product_name,...,spending_flag,order_frequency,frequency_flag,gender,state,age,date_joined,number_of_dependants,family_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Low Spender,20.5,Non-frequenct customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Low Spender,20.5,Non-frequenct customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Low Spender,20.5,Non-frequenct customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Low Spender,20.5,Non-frequenct customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Low Spender,20.5,Non-frequenct customer,Female,Alabama,31,2/17/2019,3,married,40423


#### 3. Create Regional Segmentation

In [11]:
# Create 'region' column based on 'state' column

result = []

for value in df["state"]:
    if value in ['Maine', 'New Hampshire' , 'Vermont' , 'Massachusetts' , 'Rhode Island' , 'Connecticut' , 'New York' , 'Pennsylvania' , 'New Jersey']:
        result.append("Northeast")
    elif value in ['Wisconsin' , 'Michigan' , 'Illinois' , 'Indiana' , 'Ohio' , 'North Dakota' , 'South Dakota' , 'Nebraska' , 'Kansas' , 'Minnesota' , 'Iowa' , 'Missouri']:
        result.append("Midwest")
    elif value in ['Delaware' , 'Maryland' , 'District of Columbia' , 'Virginia' , 'West Virginia' , 'North Carolina' , 'South Carolina' , 'Georgia' , 'Florida' , 'Kentucky' , 'Tennessee' , 'Mississippi' , 'Alabama' , 'Oklahoma' , 'Texas' , 'Arkansas' , 'Louisiana']:
        result.append("South")
    else:
        result.append("West")
        

In [12]:
df['region'] = result

In [15]:
# Checking output

df[['order_id', 'state', 'region']].head()

,order_id,state,region
0,2539329,Alabama,South
1,2398795,Alabama,South
2,473747,Alabama,South
3,2254736,Alabama,South
4,431534,Alabama,South


In [16]:
df['region'].value_counts(dropna = False)

region
South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: count, dtype: int64

In [19]:
# Checking the spending habits across the region

crosstab_spender = pd.crosstab(df['region'], df['spending_flag'], dropna = False)

crosstab_spender

spending_flag,High Spender,Low Spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


##### Based on the above chart, South region has both the most of highest spender and lowest spender compared to others. While the Northeast area has the lowest count in high spender customer.

In [20]:
crosstab_loyal = pd.crosstab(df['region'], df['loyalty_flag'], dropna = False)

crosstab_loyal

loyalty_flag,Loyal Customer,New Customer,Regular Customer
region,,,
Midwest,2373774,1472573,3750978
Northeast,1841785,1100207,2780744
South,3405844,2074410,5311631
West,2662690,1596800,4033423


##### In saying that, we can also see that the South region has the most orders placed. The loyalty flag is counted based on the number of the customer has placed.

#### 4. Create Exclusion Flag for Low-Activity Customers

In [26]:
# Create a column for exclusion flag for customers with less than 5 orders

df.loc[df['max_order'] < 5, 'exclusion_flag'] = 'low activity'

In [27]:
df.loc[df['max_order'] >= 5, 'exclusion_flag'] = 'high activity'

In [30]:
# Frequecny check

df['exclusion_flag'].value_counts(dropna = False)

exclusion_flag
high activity    30964564
low activity      1440295
Name: count, dtype: int64

In [31]:
# Seperate low activity customers data from the df dataset

df_low_activity = df[df['exclusion_flag'] == 'low activity']

In [36]:
df_low_activity.shape

(1440295, 33)

In [32]:
# Export low activity data customers

df_low_activity.to_pickle(os.path.join(path, '02 Data' , 'Prepared Data' , 'df_low_activity.pkl' ))

In [33]:
# Seperate high activity customers data from the df dataset

df_2 = df[df['exclusion_flag'] == 'high activity']

In [37]:
df_2.shape

(30964564, 33)

In [34]:
# Export high activity data customers

df_2.to_pickle(os.path.join(path, '02 Data' , 'Prepared Data' , 'df_high_activity.pkl' ))